# Building a Chatbot

In this project, we will build a chatbot using conversations from Cornell University's [Movie Dialogue Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). The main features of our model are LSTM cells, a bidirectional dynamic RNN, and decoders with attention. 

The conversations will be cleaned rather extensively to help the model to produce better responses. As part of the cleaning process, punctuation will be removed, rare words will be replaced with "UNK" (our "unknown" token), longer sentences will not be used, and all letters will be in the lowercase. 

With a larger amount of data, it would be more practical to keep features, such as punctuation. However, I am using FloydHub's GPU services and I don't want to get carried away with too training for too long.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import numpy as np
import tensorflow as tf
import time
import os

#Local libraries
import metrics
import loss_functions
import embedding_models

SEED = 1
np.random.seed(SEED)
tf.set_random_seed(SEED)


tf.__version__

'1.7.0'

## Load and Preprocess Data

In [3]:
corpus_dir = os.path.join("corpora")
var_names = ["train_prompts", "train_answers", "valid_prompts", "valid_answers", "vocab"]
file_names = [os.path.join(corpus_dir, var_name + ".txt") for var_name in var_names]

for (file_name, var_name) in zip(file_names, var_names):
    with open(file_name, "r", encoding="utf-8") as r:
        text = [ [token for token in line.strip().split(" ")] for line in r.readlines()]
        exec("{} = {}".format(var_name, text))
        


In [4]:
vocab2int = {pair[0]:int(pair[1]) for pair in vocab}
int2vocab = {index:word for (word, index) in vocab2int.items()}
(questions_vocab_to_int, questions_int_to_vocab) = (vocab2int, int2vocab)
(prompts_vocab_to_int, prompts_int_to_vocab) = (vocab2int, int2vocab) #Alternative names to ease the transition


(answers_vocab_to_int, answers_int_to_vocab) = (vocab2int, int2vocab)

UNK = vocab[0][0]
METATOKEN_INDEX = len(vocab2int)
META = "<META>"
EOS = "<EOS>"
PAD = "<PAD>"
GO = "<GO>"
codes = [META, EOS, PAD, GO]


In [6]:
train_prompts_int = [text_to_int(prompt, questions_vocab_to_int) for prompt in train_prompts]
train_answers_int = [text_to_int(answer, answers_vocab_to_int) for answer in train_answers]
valid_prompts_int = [text_to_int(prompt, questions_vocab_to_int) for prompt in valid_prompts]
valid_answers_int = [text_to_int(answer, answers_vocab_to_int) for answer in valid_answers]

In [7]:
for i in range(10):
    print(train_prompts[i])
    print(train_prompts_int[i])
for i in range(10):
    print(train_answers[i])
    print(train_answers_int[i])

['early', ',', 'just', 'think', '.']
[758, 2, 40, 60, 1]
['i', 'never', 'knew', 'a', 'more', '<UNK>', 'man', 'than', 'you', '.']
[4, 103, 247, 10, 113, 0, 101, 161, 3, 1]
['i', 'thought', 'you', 'we', 'are', 'above', 'all', 'that']
[4, 138, 3, 19, 14, 1658, 42, 13]
['yeah', ',', 'he', 'shall', 'really', 'go', 'for', 'that', '.']
[76, 2, 24, 263, 107, 65, 27, 13, 1]
['what', '?']
[16, 5]
['i', 'do', 'not', '<UNK>', "'", 'it', ',', 'tom', '.']
[4, 12, 9, 0, 70, 11, 2, 683, 1]
['we', 'have', 'a', 'baby', '?']
[19, 20, 10, 301, 5]
['we', 'are', 'not', 'stealing', '.', 'we', 'are', 'borrowing', '.']
[19, 14, 9, 2229, 1, 19, 14, 8034, 1]
['look', ',', 'what', 'you', 'said', 'yesterday', '.', 'i', 'guess', 'that', 'was', 'right', '.', 'it', 'is', 'not', 'worth', 'anything', 'if', 'we', 'do', 'not', 'make', 'it', 'together', '.']
[99, 2, 16, 3, 117, 693, 1, 4, 194, 13, 32, 64, 1, 11, 8, 9, 609, 134, 53, 19, 12, 9, 109, 11, 307, 1]
['i', 'said', 'nobody', 'dies', '.', 'nobody', 'dies', '.']
[4,

## Word2Vec Embeddings

In [8]:
combined_corpus = train_prompts + train_answers + valid_prompts + valid_answers
len(combined_corpus)

417780

In [9]:
combined_corpus[:5]

[['early', ',', 'just', 'think', '.'],
 ['i', 'never', 'knew', 'a', 'more', '<UNK>', 'man', 'than', 'you', '.'],
 ['i', 'thought', 'you', 'we', 'are', 'above', 'all', 'that'],
 ['yeah', ',', 'he', 'shall', 'really', 'go', 'for', 'that', '.'],
 ['what', '?']]

In [10]:
from gensim.models import Word2Vec
embedding_size = 1024
model = Word2Vec(sentences=combined_corpus, size=embedding_size, window=5, min_count=1, workers=4, sg=0)

In [11]:
model.wv['well'].shape

(1024,)

In [12]:
wordVecs = model.wv

In [13]:
word_vecs = np.zeros((len(model.wv.vocab),1024))
for i,word in enumerate(model.wv.index2word):
        word_vecs[vocab2int[word]] = model[word]
      

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Vocabulary lengths
12001
12001
12001
12001
12001
12001


In [15]:
np.save('word_Vecs.npy',word_vecs)

### Affective Embeddings

In [17]:
#Choose from embedding_models.[appended_vad, counterfitted, retrofitted]
embedding_model_path = embedding_models.appended_vad(model, vocab2int, exclude=[UNK, "patio"], regenerate=True)

12001 words to process for matching.
1000 words processed for matching.
2000 words processed for matching.
3000 words processed for matching.
4000 words processed for matching.
5000 words processed for matching.
6000 words processed for matching.
7000 words processed for matching.
8000 words processed for matching.
9000 words processed for matching.
10000 words processed for matching.
11000 words processed for matching.
12000 words processed for matching.


Neutral Vector Assigned: .
Neutral Vector Assigned: ,
Neutral Vector Assigned: you
Neutral Vector Assigned: i
Neutral Vector Assigned: ?
Neutral Vector Assigned: <UNK>
Neutral Vector Assigned: the
Neutral Vector Assigned: to
VAD Values Assigned: is --> be
Neutral Vector Assigned: not
Neutral Vector Assigned: a
Neutral Vector Assigned: it
VAD Values Assigned: do --> do
Neutral Vector Assigned: that
VAD Values Assigned: are --> be
Neutral Vector Assigned: and
Neutral Vector Assigned: what
Neutral Vector Assigned: of
Neutral Vector Assigned: !
Neutral Vector Assigned: we
VAD Values Assigned: have --> have
Neutral Vector Assigned: in
Neutral Vector Assigned: me
VAD Values Assigned: am --> be
Neutral Vector Assigned: he
Neutral Vector Assigned: -
Neutral Vector Assigned: this
Neutral Vector Assigned: for
VAD Values Assigned: know --> know
VAD Values Assigned: will --> will
Neutral Vector Assigned: your
Neutral Vector Assigned: no
VAD Values Assigned: was --> be
Neutral Vector Assigned: my
V

Neutral Vector Assigned: while
Neutral Vector Assigned: once
VAD Values Assigned: saying --> say
VAD Values Assigned: stuff --> stuff
Neutral Vector Assigned: ago
Neutral Vector Assigned: few
VAD Values Assigned: dad --> dad
VAD Values Assigned: start --> start
VAD Values Assigned: baby --> baby
VAD Values Assigned: men --> man
VAD Values Assigned: head --> head
VAD Values Assigned: play --> play
Neutral Vector Assigned: probably
VAD Values Assigned: hundred --> hundred
Neutral Vector Assigned: together
Neutral Vector Assigned: today
Neutral Vector Assigned: since
Neutral Vector Assigned: most
VAD Values Assigned: alone --> alone
VAD Values Assigned: meet --> meet
Neutral Vector Assigned: somebody
Neutral Vector Assigned: nobody
VAD Values Assigned: crazy --> crazy
VAD Values Assigned: hard --> hard
VAD Values Assigned: days --> day
VAD Values Assigned: afraid --> afraid
VAD Values Assigned: forget --> forget
VAD Values Assigned: school --> school
VAD Values Assigned: took --> take
VAD

VAD Values Assigned: daughter --> daughter
VAD Values Assigned: cold --> cold
VAD Values Assigned: leaving --> leave
VAD Values Assigned: wonderful --> wonderful
Neutral Vector Assigned: rather
VAD Values Assigned: fifty --> fifty
VAD Values Assigned: child --> child
VAD Values Assigned: kidding --> kid
VAD Values Assigned: calling --> call
VAD Values Assigned: tired --> tired
Neutral Vector Assigned: worth
VAD Values Assigned: picture --> picture
VAD Values Assigned: food --> food
Neutral Vector Assigned: fast
VAD Values Assigned: touch --> touch
VAD Values Assigned: expect --> expect
VAD Values Assigned: company --> company
VAD Values Assigned: king --> king
VAD Values Assigned: questions --> question
VAD Values Assigned: piece --> piece
VAD Values Assigned: pull --> pull
VAD Values Assigned: special --> special
VAD Values Assigned: hair --> hair
Neutral Vector Assigned: i.
VAD Values Assigned: learn --> learn
Neutral Vector Assigned: ]
VAD Values Assigned: air --> air
VAD Values Ass

VAD Values Assigned: offer --> offer
VAD Values Assigned: realize --> realize
VAD Values Assigned: killer --> killer
VAD Values Assigned: staying --> stay
VAD Values Assigned: owe --> owe
VAD Values Assigned: afternoon --> afternoon
VAD Values Assigned: service --> service
VAD Values Assigned: court --> court
VAD Values Assigned: security --> security
VAD Values Assigned: pain --> pain
VAD Values Assigned: bag --> bag
VAD Values Assigned: lawyer --> lawyer
Neutral Vector Assigned: jake
VAD Values Assigned: bastard --> bastard
VAD Values Assigned: nervous --> nervous
VAD Values Assigned: jail --> jail
VAD Values Assigned: picked --> pick
Neutral Vector Assigned: eddie
VAD Values Assigned: pictures --> pictures
VAD Values Assigned: bought --> buy
VAD Values Assigned: wake --> wake
Neutral Vector Assigned: neither
VAD Values Assigned: forgot --> forget
VAD Values Assigned: wrote --> write
Neutral Vector Assigned: instead
VAD Values Assigned: college --> college
VAD Values Assigned: grand 

...

VAD Values Assigned: tabloids --> tabloid
Neutral Vector Assigned: fanucci
Neutral Vector Assigned: simms
VAD Values Assigned: transaction --> transaction
VAD Values Assigned: practices --> practice
Neutral Vector Assigned: beynon
VAD Values Assigned: spa --> spa
Neutral Vector Assigned: laughin
VAD Values Assigned: efficient --> efficient
Neutral Vector Assigned: starbucks
VAD Values Assigned: scouting --> scout
VAD Values Assigned: pretends --> pretend
VAD Values Assigned: dandy --> dandy
VAD Values Assigned: overlooked --> overlook
Neutral Vector Assigned: leer
VAD Values Assigned: stripper --> stripper
Neutral Vector Assigned: merk
Neutral Vector Assigned: jules
Neutral Vector Assigned: footsteps
Neutral Vector Assigned: replicators
VAD Values Assigned: sunglasses --> sunglasses
VAD Values Assigned: jumps --> jump
Neutral Vector Assigned: inez
VAD Values Assigned: cables --> cable
VAD Values Assigned: suspension --> suspension
VAD Values Assigned: dome --> dome
VAD Values Assigned:

Neutral Vector Assigned: idaho
VAD Values Assigned: meth --> meth
VAD Values Assigned: blankets --> blanket
VAD Values Assigned: psyched --> psych
Neutral Vector Assigned: 'We
VAD Values Assigned: lynch --> lynching
VAD Values Assigned: hypnotized --> hypnotize
Neutral Vector Assigned: carruthers
VAD Values Assigned: unlimited --> unlimited
VAD Values Assigned: damp --> damp
Neutral Vector Assigned: limitations
VAD Values Assigned: generated --> generate
VAD Values Assigned: improving --> improve
Neutral Vector Assigned: saigon
VAD Values Assigned: hostility --> hostility
VAD Values Assigned: merchant --> merchant
Neutral Vector Assigned: singh
VAD Values Assigned: chopping --> chop
Neutral Vector Assigned: dyson
VAD Values Assigned: peeing --> pee
VAD Values Assigned: midst --> midst
Neutral Vector Assigned: i-you
VAD Values Assigned: agencies --> agency
VAD Values Assigned: utmost --> utmost
VAD Values Assigned: beeper --> beeper
VAD Values Assigned: rattling --> rattle
Neutral Vecto

VAD Values Assigned: stairwell --> stairwell
VAD Values Assigned: boxing --> boxing
VAD Values Assigned: stitch --> stitch
Neutral Vector Assigned: could'a
VAD Values Assigned: malaria --> malaria
VAD Values Assigned: baths --> bath
VAD Values Assigned: pastry --> pastry
VAD Values Assigned: nauseous --> nauseous
VAD Values Assigned: biology --> biology
VAD Values Assigned: big-time --> big
VAD Values Assigned: fallout --> fallout
VAD Values Assigned: treasurer --> treasurer
VAD Values Assigned: profanity --> profanity
VAD Values Assigned: musketeers --> musketeer
VAD Values Assigned: bollocks --> bollocks
VAD Values Assigned: imperative --> imperative
VAD Values Assigned: defended --> defend
Neutral Vector Assigned: vera
Neutral Vector Assigned: grammoo
VAD Values Assigned: diabetes --> diabetes
VAD Values Assigned: cello --> cello
VAD Values Assigned: fisherman --> fisherman
VAD Values Assigned: conditioner --> conditioner
Neutral Vector Assigned: meg
VAD Values Assigned: sickening -

VAD Values Assigned: lark --> lark
VAD Values Assigned: parting --> parting
VAD Values Assigned: entertained --> entertain
VAD Values Assigned: bind --> bind
VAD Values Assigned: scoundrel --> scoundrel
VAD Values Assigned: recite --> recite
Neutral Vector Assigned: kinder
Neutral Vector Assigned: them-
VAD Values Assigned: nobles --> noble
Neutral Vector Assigned: 1967
Neutral Vector Assigned: taiwan
VAD Values Assigned: sophomore --> sophomore
VAD Values Assigned: implicate --> implicate
VAD Values Assigned: siren --> siren
VAD Values Assigned: diversion --> diversion
VAD Values Assigned: weirder --> weird
VAD Values Assigned: supports --> support
Neutral Vector Assigned: kaminski
Neutral Vector Assigned: thaddeus
VAD Values Assigned: vultures --> vulture
VAD Values Assigned: geeks --> geek
VAD Values Assigned: infantry --> infantry
VAD Values Assigned: sublime --> sublime
VAD Values Assigned: expressing --> express
VAD Values Assigned: cupboard --> cupboard
Neutral Vector Assigned: 

8252/12001 words assigned corresponsing VAD values.
3749/12001 words assigned the neutral VAD vector.


## Additional Preprocessing

[['shut', 'your', 'mouth', '.', '<EOS>'], ['me', '?', 'well', ',', 'i', 'believe', 'in', 'rome', '.', 'you', 'would', 'have', 'to', 'after', 'what', 'i', 'have', 'seen', ',', 'how', 'people', 'outside', 'the', 'empire', 'treat', 'each', 'other', '.', '<EOS>'], ['you', 'know', 'what', 'they', 'say', '<EOS>'], ['it', 'is', 'all', 'up', 'to', 'kathryn', '.', 'amity', "'s", 'fine', 'with', 'me', 'if', 'it', 'is', 'fine', 'with', 'her', '.', '<EOS>'], ['ssh', '.', '<EOS>']]
[[362, 30, 673, 1, 12001], [22, 5, 63, 2, 4, 162, 21, 2190, 1, 3, 45, 20, 7, 176, 16, 4, 20, 253, 2, 55, 122, 552, 6, 2322, 1345, 437, 154, 1, 12001], [3, 28, 16, 38, 90, 12001], [11, 8, 42, 58, 7, 4511, 1, 8568, 48, 206, 36, 22, 53, 11, 8, 206, 36, 69, 1, 12001], [8921, 1, 12001]]


## Model

## Decoding

In [24]:
#FLAGS
flag_affect_function = True
flag_vad_values = True # Set to true if using VAD values appended onto existing embeddings
affect_function = "max_affective_content" #Other valid values "max_affective_dissonance", "min_affective_dissonance"

#Settings used by Asghar et al.
rnn_size = 1024
num_layers = 1
attention_size = 256
epochs_before_affective_loss = 40
epochs = 50
train_batch_size = 64
lambda_param_max_affective_content = 0.5
lambda_param_min_affective_dissonance=0.5
lambda_param_max_affective_dissonance = 0.4

#Training
learning_rate = 0.0001
keep_probability = 0.75

#Validation
valid_batch_size = 64


wordVecs = np.load(embedding_model_path).astype(np.float32)

embedding_size = wordVecs.shape[1] #Dynamically determine embedding size from loaded embedding file

metatoken_embedding = np.zeros((1, embedding_size), dtype=wordVecs.dtype)
wordVecsWithMeta = np.concatenate( (wordVecs, metatoken_embedding), axis=0 )
vocab_size_with_meta = wordVecsWithMeta.shape[0]

print("vocab_size_with_meta =", vocab_size_with_meta)
print("METATOKEN_INDEX =", METATOKEN_INDEX)
print("wordVecsWithMeta.shape =", wordVecsWithMeta.shape)
print("wordVecsWithMeta[METATOKEN_INDEX] =", wordVecsWithMeta[METATOKEN_INDEX])


vocab_size_with_meta = 12002
METATOKEN_INDEX = 12001
wordVecsWithMeta.shape = (12002, 1027)
wordVecsWithMeta[METATOKEN_INDEX] = [0. 0. 0. ... 0. 0. 0.]


In [24]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()


#Determines whether we use a normal loss function or an affective loss function
train_affect = tf.placeholder(tf.bool, shape=(), name="train_affect")



batch_size = tf.shape(input_data)[0]

full_embeddings = tf.Variable(wordVecsWithMeta,trainable=False,name="Weight")
enc_embed_input = tf.nn.embedding_lookup(full_embeddings, input_data)
dec_embed_input = tf.nn.embedding_lookup(full_embeddings, process_decoding_input(targets, batch_size))

output_layer = tf.layers.Dense(vocab_size_with_meta,bias_initializer=tf.zeros_initializer(),activation=tf.nn.relu)



# Find the shape of the input data for sequence_loss
with tf.name_scope("optimization"): 
    
    mask = tf.sequence_mask(target_lengths, dtype=tf.float32)
    xent = loss_functions.cross_entropy(train_logits, targets, mask)
    perplexity = tf.contrib.seq2seq.sequence_loss(train_logits, targets, mask, metrics.perplexity)
    
    
    
    vad_values = full_embeddings[:, -3:]
    input_vad_values =enc_embed_input[:,:,1024:1027]

    if flag_affect_function:
        if affect_function == "max_affective_content":
            assert flag_vad_values
            emot_embeddings = full_embeddings[:, -3:]
            neutral_vector = tf.constant([5.0, 1.0, 5.0], dtype=tf.float32)
            affective_loss = loss_functions.max_affective_content(lambda_param_max_affective_content,
                                                                train_logits, targets,
                                                                emot_embeddings,neutral_vector, mask)
        else:
            if flag_vad_values:
                emot_embeddings = full_embeddings[:, -3]
                enc_emot_embed_input = enc_embed_input[:, -3]
            else:
                emot_embeddings = full_embeddings
                enc_emot_embed_input = enc_embed_input
            
            if affect_function == "max_affective_dissonance":
                affective_loss = loss_functions.max_affective_dissonance(lambda_param_max_affective_dissonance,
                                                                train_logits, targets, 
                                                                 enc_emot_embed_input,emot_embeddings, mask)
            else:
                affective_loss = loss_functions.min_affective_dissonance(lambda_param_min_affective_dissonance,
                                                                train_logits, targets, 
                                                                 enc_emot_embed_input,emot_embeddings, mask)
    else:
        affective_loss = xent #Just so we have a term for below 
    
    train_cost = tf.cond(train_affect, true_fn=lambda: affective_loss, false_fn=lambda: xent)



Instructions for updating:
Use the retry module or similar alternatives.


### Subroutines for Sampling Output

### Training Loop Options

In [29]:
#TRAINING
display_step = 100 # Check training loss after every 100 batches

#VALIDATION
validation_check = ((len(train_prompts))//train_batch_size//2)-1 #Check validation loss every half-epoch
#Minimum number of epochs before we start checking sample output
min_epochs_before_validation = 2

#Used to make uniquely directories, not to identify when a model is saved
time_string = time.strftime("%b%d_%H:%M:%S")

checkpoint_dir = os.path.join("checkpoints", time_string)
if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir)
checkpoint_best = str(checkpoint_dir) + "/" + "best_model.ckpt" 
checkpoint_latest = str(checkpoint_dir) + "/" + "latest_model.ckpt"

log_dir = os.path.join("logging", time_string)
if not os.path.exists(log_dir): os.makedirs(log_dir)
train_log = os.path.join(log_dir, "train.csv")
valid_log = os.path.join(log_dir, "valid.csv")



### Logging Progress

In [ ]:
def question_to_seq(question, vocab_to_int, int_to_vocab):
    '''Prepare the question for the model'''
    cleaned_question = Corpus.clean_sequence(question)
    return [vocab_to_int.get(word, vocab_to_int[UNK]) for word in cleaned_question]


In [30]:
def log_entries(csv_path, *fields, header = False):
    if len(fields[0]) < 1: return
    mode = "w" if header else "a"
    with open(csv_path, mode, encoding="utf-8") as log:
        lines = []
        num_lines = len(fields[0])
        lines = "\n".join(",".join([str(field[i]) for field in fields]) 
                          for i in range(num_lines)
        )
        log.write(lines + "\n")

In [ ]:
# Use a question from the data as your input
random = np.random.choice(len(train_prompts_int))
prompt_int = train_prompts_int[random]
answer_int = train_answers_int[random]

saver = tf.train.Saver()
with tf.Session() as sess:
    # Run the model with the input question
    saver.restore(sess, checkpoint)
    check_response(sess, prompt_int, answer_int, best_only=False)
    


In [31]:
def clear_fields(log_fields):
    for field in log_fields:
        field.clear()